In [13]:
import torch
import torch.nn as nn
import torch.nn.functional
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

rm_quoted = re.compile(r"\".*?\"")
arrStopwords = stopwords.words("english")
arrStopwords.extend(["br", "<br>", "<br />"])

In [6]:
data = pd.read_csv("./IMDB Dataset.csv")

In [7]:
countvec = CountVectorizer(stop_words=arrStopwords)

In [8]:
data.iloc[4]["review"]

'Petter Mattei\'s "Love in the Time of Money" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait about human relations. This is a movie that seems to be telling us what money, power and success do to people in the different situations we encounter. <br /><br />This being a variation on the Arthur Schnitzler\'s play about the same theme, the director transfers the action to the present time New York where all these different characters meet and connect. Each one is connected in one way, or another to the next person, but no one seems to know the previous point of contact. Stylishly, the film has a sophisticated luxurious look. We are taken to see how these people live and the world they live in their own habitat.<br /><br />The only thing one gets out of all these souls in the picture is the different stages of loneliness each one inhabits. A big city is not exactly the best place in which human relations find sincere fulfillment, as one discerns is the case wit

In [9]:
mdata = data

for i in range(len(data)):
    mdata.iloc[i]["review"] = re.sub(rm_quoted, "", mdata.iloc[i]["review"])

vectorized_data = countvec.fit_transform([review for review in mdata["review"]])

for i in range(len(data)):
        mdata.iloc[i]["review"] = vectorized_data[i]                    # .todense() # для конвертации в плотную матрицу (и паники ядра, лол)

mdata.replace(to_replace={"sentiment": {"negative": 0, "positive": 1}}, inplace=True)

mdata.head()

,review,sentiment
0,"(0, 62294)\t1\n (0, 73360)\t1\n (0, 55939)...",1
1,"(0, 62294)\t1\n (0, 95490)\t1\n (0, 76515)...",1
2,"(0, 62294)\t1\n (0, 95490)\t1\n (0, 73753)...",1
3,"(0, 32619)\t1\n (0, 95483)\t1\n (0, 48252)...",0
4,"(0, 62294)\t6\n (0, 30123)\t1\n (0, 87877)...",1


In [10]:
type(data.iloc[4]["review"])

scipy.sparse._csr.csr_matrix

In [11]:
mdata.shape

(50000, 2)

In [15]:
train = mdata[:40000]
test = mdata[40000:]

train_dl = DataLoader(train, shuffle=True, batch_size=50, drop_last=True)
test_dl = DataLoader(test, shuffle=True, batch_size=50, drop_last=True)

In [17]:
class BiLSTM_SentimentAnalysis(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim, dropout) :

        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim, 3)

    def forward(self, x, hidden):

        embeddings = self.embedding(x)
        outputs, hidden = self.lstm(embeddings, hidden)
        outputs = self.dropout(outputs)
        outputs = self.fc(outputs)
        outputs = outputs[:, -1]
        return outputs, hidden
    
    def init_hidden(self):                  # Инициализация скрытого слоя LSTM
        return (torch.zeros(1, 50, 32), torch.zeros(1, 50, 32))

In [19]:
imdb = BiLSTM_SentimentAnalysis(len(word2index), 64, 32, 0.2)

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(imdb.parameters(), lr = 3e-4)

NameError: name 'word2index' is not defined